## MonarQ's transpiler

MonarQ's transpiler is typically used behind the scene when you use the "monarq.default" device, but you can also use it in a standalone manner if need be. 

In [2]:
from pennylane_snowflurry.transpiler.monarq_transpile import Transpiler
from pennylane_snowflurry.transpiler.transpiler_config import MonarqDefaultConfig
from pennylane.tape import QuantumTape
import pennylane as qml

In [5]:
# create a QuantumTape (this is one way to do it, but you can also retrieve it from a QNode)

ops = [qml.Hadamard(0), qml.CNOT([0, 1])]
measurements = [qml.counts(wires=[0, 1])]
shots = 1000

tape = QuantumTape(ops=ops, measurements=measurements, shots=shots)

tape.operations


[H(0), CNOT(wires=[0, 1])]

In [6]:
# configure and run the transpiler

config = MonarqDefaultConfig(use_benchmark=False)
transpiler = Transpiler.get_transpiler(config)

# since the transpiler function is a transform, you have to index [0][0] in order to get the resulting tape
tape = transpiler(tape)[0][0]

tape.operations

[X90(wires=[0]), X90(wires=[4]), Z(4), CZ(wires=[0, 4]), X90(wires=[4])]

## Steps

The default transpiling process contained in ```MonarqDefaultConfig``` contains the following steps :
1. **decompose** the circuit operations in a subset of operations (for simplifying subsequent operations)
2. **map** the circuit to physical qubits in MonarQ's topology
3. **route** 2 qubits gates which are not connected with respect to the mapping
4. **simplify** the circuit by commuting gates, merging rotations and cancelling inverses and trivial gates
5. **decompose** the circuit a subset of operations which are native to MonarQ

All transpiling steps reside in ```pennylane_snowflurry.transpiler.steps.*```

Steps are defined using a base class called ```BaseStep```. 

This class has two children, which subdivide the steps in two categories : 
- ```PreProcessing``` : steps that are applied before execution on the machine. Typically affects the circuit (through the ```QuantumTape``` class)
- ```PostProcessing``` : steps that are applied after execution on the machine. Typically affects the results.

It is possible to define new transpiling or postprocessing steps by overriding the ```PreProcessing```or ```PostProcessing``` classes as so : 

In [ ]:
from pennylane_snowflurry.transpiler.steps.interfaces.pre_processing import PreProcStep
from pennylane_snowflurry.transpiler.steps.interfaces.post_processing import PostProcStep

# this step will be executed in the transpiler
class MyPreProcessingStep(PreProcStep):
    def execute(self, tape):
        # define your own behaviour here
        return super().execute(tape) # return your modified tape here

# this step will be executed in the post-processor
class MyPostProcessingStep(PostProcStep):

    def execute(self, tape : QuantumTape, results : dict[str, int]):
        return super().execute(tape, results)